# Denoise
We may want to do some basic denoising to clean up the image

In [ ]:
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,
                                 denoise_wavelet, estimate_sigma)

In [ ]:
mosaic_hdu = fits.open('mosaic.fits')[0]
denoised = denoise_tv_chambolle(mosaic_hdu.data, weight=0.1, multichannel=True)

In [ ]:
mosaic_hdu.data = denoised
mosaic_hdu.writeto('denoised.fits', overwrite=True)